In [1]:
!pip install -q pyomo

**Ex 3 part 4** \\
Let $x_i's$ be the number of drones of model i, where $i\in \{1,2,3,4,5\}$ and $x_1$ is for number of drones of MANAS, $x_2$ is for JAY, $x_3$ is for VEER, $x_4$ is for SPASHT and $x_5$ is for DHAKSH. \\
Now, as we want to maximize total carrying capacity. Hence, the Optimization model will be as follows \\
$Maximize \ \  12x_1+14x_2+10x_3+15x_4+14x_5\\
\text{s.t.} \ \ \ \ \ \ \ \ \ \ \ 6x_1+7.5x_2+9.5x_3+10x_4+12.5x_5 \leq 59 \\
\ \ \ \ \ \ 4.25x_1+3.25x_2+2.5x_3+3.45x_4+2.25x_5 \leq 73 \\
\ \ \ \ \ \ and\ x_i's\ will\ be\ non\ negative\ integers$

In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
data_csvfile = pd.read_csv('coeff_csv.csv')

In [6]:
data_csvfile.columns

Index(['model', 'capacity', 'cost', 'parkingspace'], dtype='object')

In [7]:
data_csvfile['model']

0     MANAS
1       JAY
2      VEER
3    SPASHT
4    DHAKSH
Name: model, dtype: object

In [8]:
data_csvfile.index

RangeIndex(start=0, stop=5, step=1)

In [9]:
len(data_csvfile.index)

5

In [10]:
# create a model
model1_lab4_ex3 = ConcreteModel()

In [11]:
# here N denotes no. of variables
N = len(data_csvfile.index) 

In [12]:
M = 2 #no of constraints

In [13]:
col_indices = range(N)

In [14]:
#declaring the decision variables in the model
model1_lab4_ex3.x = Var(col_indices,domain=NonNegativeIntegers)

In [15]:
#objection function
model1_lab4_ex3.objective = Objective(expr=summation(data_csvfile['capacity'], model1_lab4_ex3.x),sense=maximize)

In [16]:
model1_lab4_ex3.constraints = ConstraintList()

In [17]:
#Now we can add the cost constraint using the data obtained from the csv file
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 59)

In [18]:
#Now we can add the parking space constraint using the data obtained from the csv file
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 73)

In [19]:
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 

In [20]:
!apt-get install -y -qq coinor-cbc

In [21]:
opt_cbc = SolverFactory('cbc')

In [22]:
result = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


**Ex 3 part 6**

In [23]:
# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()


Objective =  114.0

Decision Variables
Num of drone  MANAS 6.0
Num of drone  JAY 3.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=2
    Key : Lower : Body  : Upper
      1 :  None :  58.5 :  59.0
      2 :  None : 35.25 :  73.0


In [24]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeReals

In [25]:
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   6.0 :  None : False : False : NonNegativeReals
          1 :     0 :   3.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4]

1 Cons

In [26]:
result1 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


**Ex 3 part 7**

In [27]:
# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()


Objective =  117.9999996

Decision Variables
Num of drone  MANAS 9.8333333
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=2
    Key : Lower : Body         : Upper
      1 :  None :   58.9999998 :  59.0
      2 :  None : 41.791666525 :  73.0


**Ex 3 part 8**

No the solutions of MILP cannot be obtained by merely rounding of the solution of LP. Because the LP will try to assign max possible value(if allowed by parking constraint) to those model which has highest capacity to cost ratio and all other 0.

In [28]:
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 71)
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 87)

In [29]:
model1_lab4_ex3.constraints[1].deactivate()
model1_lab4_ex3.constraints[2].deactivate()

In [30]:
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 9.8333333 :  None : False : False : NonNegativeReals
          1 :     0 :       0.0 :  None : False : False : NonNegativeReals
          2 :     0 :       0.0 :  None : False : False : NonNegativeReals
          3 :     0 :       0.0 :  None : False : False : NonNegativeReals
          4 :     0 :       0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[

**Ex 3 part 9**

Solving new LP

In [31]:
result2 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [32]:
# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()


Objective =  141.999996

Decision Variables
Num of drone  MANAS 11.833333
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=4
    Key : Lower : Body        : Upper
      3 :  None :   70.999998 :  71.0
      4 :  None : 50.29166525 :  87.0


The optimal value of the new LP has increases and also the no. of drones of model which was non zero in previous LP has increased. 

Solving new MILP

In [33]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeIntegers

In [34]:
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 11.833333 :  None : False : False : NonNegativeIntegers
          1 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :       0.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 

In [35]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [36]:
# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()


Objective =  138.0

Decision Variables
Num of drone  MANAS 8.0
Num of drone  JAY 3.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=4
    Key : Lower : Body  : Upper
      3 :  None :  70.5 :  71.0
      4 :  None : 43.75 :  87.0


The optimal objective value has increased and no. of drones of MANAS has increased.

**Ex 3 part 10**

**Solving for first given pair**

In [37]:
model1_lab4_ex3.constraints[3].deactivate()
model1_lab4_ex3.constraints[4].deactivate()

Solvng MILP

In [38]:
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 67)
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 88)
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   8.0 :  None : False : False : NonNegativeIntegers
          1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] 

In [39]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  132.0

Decision Variables
Num of drone  MANAS 11.0
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=6
    Key : Lower : Body  : Upper
      5 :  None :  66.0 :  67.0
      6 :  None : 46.75 :  88.0


Solving LP

In [40]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeReals

In [41]:
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  11.0 :  None : False : False : NonNegativeReals
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*

In [42]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  134.000004

Decision Variables
Num of drone  MANAS 11.166667
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=6
    Key : Lower : Body        : Upper
      5 :  None :   67.000002 :  67.0
      6 :  None : 47.45833475 :  88.0


**Solving for second given pair.**

Solving LP

In [43]:
model1_lab4_ex3.constraints[5].deactivate()
model1_lab4_ex3.constraints[6].deactivate()
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 65)
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 89)
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {1, 2, 3, 4, 5, 6, 7, 8}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 11.166667 :  None : False : False : NonNegativeReals
          1 :     0 :       0.0 :  None : False : False : NonNegativeReals
          2 :     0 :       0.0 :  None : False : False : NonNegativeReals
          3 :     0 :       0.0 :  None : False : False : NonNegativeReals
          4 :     0 :       0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*

In [44]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  129.999996

Decision Variables
Num of drone  MANAS 10.833333
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=8
    Key : Lower : Body        : Upper
      7 :  None :   64.999998 :  65.0
      8 :  None : 46.04166525 :  89.0


Solving MILP

In [45]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeIntegers

model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {1, 2, 3, 4, 5, 6, 7, 8}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 10.833333 :  None : False : False : NonNegativeIntegers
          1 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :       0.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize 

In [46]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  126.0

Decision Variables
Num of drone  MANAS 7.0
Num of drone  JAY 3.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=8
    Key : Lower : Body : Upper
      7 :  None : 64.5 :  65.0
      8 :  None : 39.5 :  89.0


**Solving for 3rd given pair.**

Solving MILP

In [47]:
model1_lab4_ex3.constraints[7].deactivate()
model1_lab4_ex3.constraints[8].deactivate()
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 63)
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 90)
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   7.0 :  None : False : False : NonNegativeIntegers
          1 :     0 :   3.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[

In [48]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  124.0

Decision Variables
Num of drone  MANAS 8.0
Num of drone  JAY 2.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=10
    Key : Lower : Body : Upper
      9 :  None : 63.0 :  63.0
     10 :  None : 40.5 :  90.0


Solving LP

In [49]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeReals

model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   8.0 :  None : False : False : NonNegativeReals
          1 :     0 :   2.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 

In [50]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  126.0

Decision Variables
Num of drone  MANAS 10.5
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=10
    Key : Lower : Body   : Upper
      9 :  None :   63.0 :  63.0
     10 :  None : 44.625 :  90.0


**Solving for 4th given pair**

Solving LP

In [51]:
model1_lab4_ex3.constraints[9].deactivate()
model1_lab4_ex3.constraints[10].deactivate()
model1_lab4_ex3.constraints.add(summation(data_csvfile['cost'], model1_lab4_ex3.x) <= 61)
model1_lab4_ex3.constraints.add(summation(data_csvfile['parkingspace'], model1_lab4_ex3.x) <= 91)
model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  10.5 :  None : False : False : NonNegativeReals
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10

In [52]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  122.000004

Decision Variables
Num of drone  MANAS 10.166667
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=12
    Key : Lower : Body        : Upper
     11 :  None :   61.000002 :  61.0
     12 :  None : 43.20833475 :  91.0


Solving MILP

In [53]:
for i in col_indices:
  model1_lab4_ex3.x[i].domain=NonNegativeIntegers

model1_lab4_ex3.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 10.166667 :  None : False : False : NonNegativeIntegers
          1 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          2 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          3 :     0 :       0.0 :  None : False : False : NonNegativeIntegers
          4 :     0 :       0.0 :  None : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   T

In [54]:
result3 = opt_cbc.solve(model1_lab4_ex3)
print('Solver status:', result3.solver.status)
print('Solver termination condition:',result3.solver.termination_condition)

# display solution
print('\nObjective = ', model1_lab4_ex3.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num of drone ',data_csvfile['model'][i], model1_lab4_ex3.x[i].value)

print('\nConstraints')
model1_lab4_ex3.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  120.0

Decision Variables
Num of drone  MANAS 10.0
Num of drone  JAY 0.0
Num of drone  VEER 0.0
Num of drone  SPASHT 0.0
Num of drone  DHAKSH 0.0

Constraints
constraints : Size=12
    Key : Lower : Body : Upper
     11 :  None : 60.0 :  61.0
     12 :  None : 42.5 :  91.0


The optimal value decreases as we move from pair 1 to pair 4, because we are decreasing the budget. And as we can see that the parking space constraint is not active any case. So, the decrease in parking space is not of concern.
hence the decrease in budget is causing decrease in optimal value.